In [1]:
import torch
import torchvision
import sys
sys.path.append('..')
from config import DATA_PATH
from torchvision.transforms import v2 as transforms

/Users/timcares/miniforge3/envs/mmrl/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import os
from data_utils import curl_dataset
import subprocess
from bpe_encoder import encode

In [2]:

data_path="../data/language/enwik9"
def prepare_data():
    os.makedirs(data_path, exist_ok=True)
    name = curl_dataset("http://mattmahoney.net/dc/enwik9.zip")
    os.system(f"unzip {name}")
    os.remove(name)
    os.system(f"perl ../setup/clean_enwik9.pl enwik9 > enwik9.txt")
    os.remove("enwik9")
    encode(f'{DATA_PATH}/encoder.json', f'{DATA_PATH}/vocab.bpe', ['enwik9.txt'], ['enwik9.bpe'], keep_empty=True)
    os.remove("enwik9.txt")
    process = ['fairseq-preprocess', '--only-source', '--srcdict', f'{DATA_PATH}/dict.txt',
                '--trainpref', 'enwik9.bpe', '--destdir', f'{data_path}', '--workers', f'10']
    subprocess.run(process)
    os.remove("enwik9.bpe")

In [16]:
prepare_data()

Archive:  enwik9.zip
  inflating: enwik9                  


In [2]:
transform = transforms.Compose(
    [
        transforms.PILToTensor(),
        transforms.Resize((224, 224)),
        transforms.ToDtype(torch.float32, scale=True),
        transforms.Normalize(
            mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]
        ),
    ]
)

In [3]:
ds = torchvision.datasets.CIFAR100(root=DATA_PATH,
                              train=True,
                              transform=transform,
                              download=True)

Files already downloaded and verified


In [4]:
data_loader = torch.utils.data.DataLoader(ds,
                                          batch_size=50_000,
                                          shuffle=False,
                                          num_workers=5)

In [5]:
data_loader_iter = iter(data_loader)

In [6]:
r=next(data_loader_iter)
r

[tensor([[[[ 2.2489,  2.2489,  2.2489,  ...,  1.2214,  1.5125,  0.9988],
           [ 2.2489,  2.2318,  2.2318,  ...,  0.7933,  0.6392,  0.3823],
           [ 2.2489,  2.2318,  2.2489,  ...,  1.1187,  0.7248, -0.0458],
           ...,
           [ 0.4166,  0.3138,  0.2796,  ..., -1.6042, -1.0048, -0.8164],
           [-0.0287, -0.0629,  0.0398,  ..., -1.7412, -0.4568,  0.2967],
           [-0.6281, -0.6109, -0.3883,  ..., -1.5357, -0.3198,  0.2453]],
 
          [[ 2.4286,  2.4286,  2.4286,  ...,  1.5532,  1.8859,  1.3606],
           [ 2.4286,  2.4111,  2.4111,  ...,  1.0455,  0.9055,  0.6604],
           [ 2.4286,  2.4111,  2.4286,  ...,  1.4482,  1.0805,  0.2927],
           ...,
           [ 1.2031,  1.1506,  1.0980,  ..., -1.7381, -0.9503, -0.6877],
           [ 0.7129,  0.6779,  0.7654,  ..., -1.7556, -0.0749,  0.7829],
           [ 0.1001,  0.1001,  0.3102,  ..., -1.4055,  0.2927,  0.9930]],
 
          [[ 2.6400,  2.6400,  2.6400,  ...,  1.5594,  1.7511,  1.1062],
           [ 

In [7]:
r[0].shape

torch.Size([50000, 3, 32, 32])